In [19]:
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer
import time
from sklearn.preprocessing import LabelEncoder

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
test['date'] = pd.to_datetime(test['date'])
train['date'] = pd.to_datetime(train['date'])

In [8]:
test.head()

,ID,date,cluster_id,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2022-01-01,2022-01-01,cluster_1,15.2,7.9,14.2,5.2,21902.986667,29784.506667,22.2,37.8,172.732245,4.173333,0.780000
1,cluster_2_2022-01-01,2022-01-01,cluster_2,15.9,4.1,15.3,1.3,23550.080000,30647.540000,22.7,38.5,163.972679,5.146667,0.730000
2,cluster_3_2022-01-01,2022-01-01,cluster_3,18.6,4.9,18.8,1.8,24179.643333,31545.103333,15.5,26.3,198.645137,5.713333,0.730000
3,cluster_4_2022-01-01,2022-01-01,cluster_4,13.3,3.9,11.9,1.3,23839.203333,30766.446667,16.1,27.4,183.517316,4.200000,0.486667
4,cluster_1_2022-01-02,2022-01-02,cluster_1,14.1,8.1,11.2,5.3,21018.193333,29847.110000,34.4,59.4,223.335840,4.030000,1.170000


In [9]:
train.head()

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2014-01-01,2014-01-01,cluster_1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333
1,cluster_2_2014-01-01,2014-01-01,cluster_2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667
2,cluster_3_2014-01-01,2014-01-01,cluster_3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667
3,cluster_4_2014-01-01,2014-01-01,cluster_4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667
4,cluster_1_2014-01-02,2014-01-02,cluster_1,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000


In [10]:
test_with_seasonal = test.copy()
train_with_seasonal = train.copy()

In [11]:
test_with_seasonal["month"] = test_with_seasonal["date"].dt.month
test_with_seasonal["dayofweek"] = test_with_seasonal["date"].dt.dayofweek
test_with_seasonal["quarter"] = test_with_seasonal["date"].dt.quarter
test_with_seasonal["day"] = test_with_seasonal["date"].dt.day
test_with_seasonal["weekofyear"] = test_with_seasonal["date"].dt.isocalendar().week.astype(int)
test_with_seasonal = test_with_seasonal.drop(columns=["date"])

train_with_seasonal["month"] = train_with_seasonal["date"].dt.month
train_with_seasonal["dayofweek"] = train_with_seasonal["date"].dt.dayofweek
train_with_seasonal["quarter"] = train_with_seasonal["date"].dt.quarter
train_with_seasonal["day"] = train_with_seasonal["date"].dt.day
train_with_seasonal["weekofyear"] = train_with_seasonal["date"].dt.isocalendar().week.astype(int)
train_with_seasonal = train_with_seasonal.drop(columns=["date"])

In [12]:
test_with_seasonal.to_csv("test_all_with_seasonal.csv", index=False)
train_with_seasonal.to_csv("train_all_with_seasonal.csv", index=False)

In [13]:
train = pd.read_csv('test_all_with_seasonal.csv')
test = pd.read_csv('train_all_with_seasonal.csv')

In [14]:
if 'cluster_id' in test.columns:
    le = LabelEncoder()
    test['cluster_id'] = le.fit_transform(test['cluster_id'])

In [15]:
if 'cluster_id' in train.columns:
    le = LabelEncoder()
    train['cluster_id'] = le.fit_transform(train['cluster_id'])

In [31]:
test.to_csv('test_all_with_seasonal_cluster.csv', index=False)
train.to_csv('train_all_with_seasonal_cluster.csv', index=False)

In [35]:
train = pd.read_csv('train_all_with_seasonal_cluster.csv')
test = pd.read_csv('test_all_with_seasonal_cluster.csv')

In [36]:
train.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,0,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333,1,2,1,1,1
1,cluster_2_2014-01-01,1,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667,1,2,1,1,1
2,cluster_3_2014-01-01,2,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667,1,2,1,1,1
3,cluster_4_2014-01-01,3,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667,1,2,1,1,1
4,cluster_1_2014-01-02,0,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000,1,3,1,2,1


In [37]:
target_col = "electricity_consumption"

In [38]:
param_dist = {
    'n_estimators': [100, 300, 500, 700, 1000],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.25, 0.5, 1, 5],
    'min_child_weight': [1, 2, 3, 5, 7, 10],
    'reg_alpha': [0, 0.001, 0.01, 0.1, 1],     # L1 regularization
    'reg_lambda': [0.1, 0.5, 1.0, 2.0, 5.0]    # L2 regularization
}

In [47]:
def train_and_eval_tuned(df_input, label):
    X = df_input.drop(columns=[target_col])
    y = df_input[target_col]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    xgb = XGBRegressor(objective="reg:squarederror", random_state=42)

    # Use RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=1000,  
        scoring='neg_root_mean_squared_error',
        cv=3,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    y_pred = best_model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"Tuned {label} RMSE: {rmse:.4f}")
    print(f"Best params: {random_search.best_params_}")

    return best_model

In [48]:
train_cluster_date = train.drop(columns=["ID"])

In [49]:
model = train_and_eval_tuned(train_cluster_date, "With seasonal and clusterid features")

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits
Tuned With seasonal and clusterid features RMSE: 27.1101
Best params: {'subsample': 0.5, 'reg_lambda': 0.5, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.5, 'colsample_bytree': 0.8}


In [50]:
X_test = test.drop(columns=["ID"]) 

In [51]:
y_test_pred = model.predict(X_test)

In [52]:
submission = pd.DataFrame({
    "ID": test["ID"],
    "electricity_consumption": y_test_pred
})
submission.to_csv("submission15_1000_verbose2.csv", index=False)